In [1]:
import pandas as pd
import numpy as np

In [2]:
threshold = "FINAL"
jula_1 = pd.read_excel(f"../temp/jula_tier_1_example_report_{threshold}.xlsx")
jula_1["tier"] = 1
jula_2 = pd.read_excel(f"../temp/jula_tier_2_example_report_{threshold}.xlsx")
jula_2["tier"] = 2
jula_3 = pd.read_excel(f"../temp/jula_tier_3_example_report_{threshold}.xlsx")
jula_3["tier"] = 3
jula_data = [jula_1, jula_2, jula_3]
jula_concat = pd.concat(jula_data)

tele2_1 = pd.read_excel(f"../temp/tele2_tier_1_example_report_{threshold}.xlsx")
tele2_1["tier"] = 1
tele2_2 = pd.read_excel(f"../temp/tele2_tier_2_example_report_{threshold}.xlsx")
tele2_2["tier"] = 2
tele2_3 = pd.read_excel(f"../temp/tele2_tier_3_example_report_{threshold}.xlsx")
tele2_3["tier"] = 3
tele2_data =[tele2_1, tele2_2, tele2_3]
tele2_concat = pd.concat(tele2_data)


nordea_1 = pd.read_excel(f"../temp/nordea_tier_1_example_report_{threshold}.xlsx")
nordea_1["tier"] = 1
nordea_2 = pd.read_excel(f"../temp/nordea_tier_2_example_report_{threshold}.xlsx")
nordea_2["tier"] = 2
nordea_3 = pd.read_excel(f"../temp/nordea_tier_3_example_report_{threshold}.xlsx")
nordea_3["tier"] = 3
nordea_data = [nordea_1, nordea_2, nordea_3]
nordea_concat = pd.concat(nordea_data)


länsförsäkringar_1 = pd.read_excel(f"../temp/länsförsäkringar_tier_1_example_report_{threshold}.xlsx")
länsförsäkringar_1["tier"] = 1
länsförsäkringar_2 = pd.read_excel(f"../temp/länsförsäkringar_tier_2_example_report_{threshold}.xlsx")
länsförsäkringar_2["tier"] = 2
länsförsäkringar_3 = pd.read_excel(f"../temp/länsförsäkringar_tier_3_example_report_{threshold}.xlsx")
länsförsäkringar_3["tier"] = 3
länsförsäkringar_data = [länsförsäkringar_1, länsförsäkringar_2, länsförsäkringar_3]
länsförsäkringar_concat = pd.concat(länsförsäkringar_data)


In [3]:
columns = list(jula_1.columns)

In [4]:
def get_data_based_on_tier(client_dfs):
    new_df = pd.DataFrame(columns=columns)
    combined_df = pd.concat(client_dfs)
    tier_mapping = {
        "high": "medium",
        "medium": "low",
        "low": "low",
    }
    for i, df in enumerate(client_dfs):
        for index, row in df.iterrows():
            if row.tier == 1:
                if row.sample_size in ("high", "medium"):
                    new_df = new_df.append(row)
            elif row.tier == 2:
                if new_df[new_df["hostname"] == row.hostname]["hostname"].count() == 0:
                    if row.sample_size == "high":
                        print("qiwjeioqwjewioqe")
                        # Degrade sample_size if lower tier
                        #row["sample_size"] = tier_mapping[row["sample_size"]]
                        row.sample_size = "medium"
                        new_df = new_df.append(row)
                    elif row.sample_size == "medium":
                        new_row = combined_df[(combined_df["tier"] == 3) & (combined_df["hostname"] == row.hostname)]
                        #new_row["sample_size"] = tier_mapping[row["sample_size"]]
                        new_row.sample_size = "medium"
                        new_row["client_fixations"] = (
                            np.floor(row["client_fixations"] * 0.25 + new_row["client_fixations"].sum() * 0.75)
                        )
                        new_df = new_df.append(new_row)
                    else:
                        new_row = combined_df[(combined_df["tier"] == 3) & (combined_df["hostname"] == row.hostname)]
                        #new_row["sample_size"] = tier_mapping[row["sample_size"]]
                        new_row.sample_size = "medium"
                        new_row["client_fixations"] = (
                            np.floor(row["client_fixations"] * 0.15 + new_row["client_fixations"].sum() * 0.85)
                        )
                        new_df = new_df.append(new_row)
            elif row.tier == 3:
                if new_df[new_df["hostname"] == row.hostname]["hostname"].count() == 0:
                    row["sample_size"] = "low"
                    new_df = new_df.append(row)
    return new_df

final_jula_df = get_data_based_on_tier(jula_data)
final_nordea_df = get_data_based_on_tier(nordea_data)
final_tele2_df = get_data_based_on_tier(tele2_data)
final_länsförsäkringar_df = get_data_based_on_tier(länsförsäkringar_data)

qiwjeioqwjewioqe


/tmp/ipykernel_29980/2766694437.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
/tmp/ipykernel_29980/2766694437.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row.sample_size = "medium"
/tmp/ipykernel_29980/2766694437.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row["client_fixations"] = (
/tmp/ipykernel_29980/2766694437.py:37: FutureWarning: The frame.append method

In [5]:
def do_math(row, arg1, arg2):
    try:
        return round(row[f"{arg1}"] / row[f"{arg2}"] * 100, 2)
    except ZeroDivisionError:
        return 0

final_jula_df["client_Inview Ratio"] = final_jula_df.apply(lambda x: round(x["client_inview"] / x["client_imps"] * 100, 2), axis=1)
final_jula_df["client_Fixation/Inview Ratio"] = final_jula_df.apply(lambda x: do_math(x, "client_fixations", "client_inview"), axis=1)
final_jula_df["tobii_Fixation/Inview Ratio"] = final_jula_df.apply(lambda x: do_math(x, "tobii_fixations", "tobii_inview"), axis=1)
final_jula_df["client_fixation/impression_ratio"] = final_jula_df.apply(lambda x: do_math(x, "client_fixations", "client_imps"), axis=1)
final_jula_df = final_jula_df[["hostname", "client_imps", "client_fixations", "tobii_fixation/impression_ratio", "client_inview", "client_Inview Ratio", "client_Fixation/Inview Ratio", "tobii_Fixation/Inview Ratio", "tobii_inview/impression_ratio", "sample_size", "tier"]]
final_jula_df = final_jula_df.rename(
    columns={
        "client_imps": "Impressions", 
        "client_fixations": "Fixations",
        "client_inview": "Inviews",
        #"tobii_inview/impression_ratio": "tobii_Inview Ratio",
        #"tobii_fixation/impression_ratio": "Tobii Fixation Ratio",
        #"client_fixation/impression_ratio": "Fixation Ratio",
        "tobii_Fixation/Inview Ratio": "Fixation/Tobii Inview Ratio",
        "client_Fixation/Inview Ratio": "Fixation/Inview Ratio",
        "sample_size": "Sample Size"
    }
)

final_nordea_df["client_Inview Ratio"] = final_nordea_df.apply(lambda x: round(x["client_inview"] / x["client_imps"] * 100, 2), axis=1)
final_nordea_df["client_Fixation/Inview Ratio"] = final_nordea_df.apply(lambda x: do_math(x, "client_fixations", "client_inview"), axis=1)
final_nordea_df["tobii_Fixation/Inview Ratio"] = final_nordea_df.apply(lambda x: do_math(x, "tobii_fixations", "tobii_inview"), axis=1)
final_nordea_df["client_fixation/impression_ratio"] = final_nordea_df.apply(lambda x: do_math(x, "client_fixations", "client_imps"), axis=1)
final_nordea_df = final_nordea_df[["hostname", "client_imps", "client_fixations", "tobii_fixation/impression_ratio", "client_inview", "client_Inview Ratio", "client_Fixation/Inview Ratio", "tobii_Fixation/Inview Ratio", "tobii_inview/impression_ratio", "sample_size", "tier"]]
final_nordea_df = final_nordea_df.rename(
    columns={
        "client_imps": "Impressions", 
        "client_fixations": "Fixations",
        "client_inview": "Inviews",
        #"tobii_inview/impression_ratio": "tobii_Inview Ratio",
        #"tobii_fixation/impression_ratio": "Tobii Fixation Ratio",
        #"client_fixation/impression_ratio": "Fixation Ratio",
        "tobii_Fixation/Inview Ratio": "Fixation/Tobii Inview Ratio",
        "client_Fixation/Inview Ratio": "Fixation/Inview Ratio",
        "sample_size": "Sample Size"
    }
)

final_tele2_df["client_Inview Ratio"] = final_tele2_df.apply(lambda x: round(x["client_inview"] / x["client_imps"] * 100, 2), axis=1)
final_tele2_df["client_Fixation/Inview Ratio"] = final_tele2_df.apply(lambda x: do_math(x, "client_fixations", "client_inview"), axis=1)
final_tele2_df["tobii_Fixation/Inview Ratio"] = final_tele2_df.apply(lambda x: do_math(x, "tobii_fixations", "tobii_inview"), axis=1)
final_tele2_df["client_fixation/impression_ratio"] = final_tele2_df.apply(lambda x: do_math(x, "client_fixations", "client_imps"), axis=1)
final_tele2_df = final_tele2_df[["hostname", "client_imps", "client_fixations", "tobii_fixation/impression_ratio", "client_inview", "client_Inview Ratio", "client_Fixation/Inview Ratio", "tobii_Fixation/Inview Ratio", "tobii_inview/impression_ratio", "sample_size", "tier"]]
final_tele2_df = final_tele2_df.rename(
    columns={
        "client_imps": "Impressions", 
        "client_fixations": "Fixations",
        "client_inview": "Inviews",
        #"tobii_inview/impression_ratio": "tobii_Inview Ratio",
        #"tobii_fixation/impression_ratio": "Tobii Fixation Ratio",
        #"client_fixation/impression_ratio": "Fixation Ratio",
        "tobii_Fixation/Inview Ratio": "Fixation/Tobii Inview Ratio",
        "client_Fixation/Inview Ratio": "Fixation/Inview Ratio",
        "sample_size": "Sample Size"
    }
)

final_länsförsäkringar_df["client_Inview Ratio"] = final_länsförsäkringar_df.apply(lambda x: round(x["client_inview"] / x["client_imps"] * 100, 2), axis=1)
final_länsförsäkringar_df["client_Fixation/Inview Ratio"] = final_länsförsäkringar_df.apply(lambda x: do_math(x, "client_fixations", "client_inview"), axis=1)
final_länsförsäkringar_df["tobii_Fixation/Inview Ratio"] = final_länsförsäkringar_df.apply(lambda x: do_math(x, "tobii_fixations", "tobii_inview"), axis=1)
final_länsförsäkringar_df["client_fixation/impression_ratio"] = final_länsförsäkringar_df.apply(lambda x: do_math(x, "client_fixations", "client_imps"), axis=1)
final_länsförsäkringar_df = final_länsförsäkringar_df[["hostname", "client_imps", "client_fixations", "tobii_fixation/impression_ratio", "client_inview", "client_Inview Ratio", "client_Fixation/Inview Ratio", "tobii_Fixation/Inview Ratio", "tobii_inview/impression_ratio", "sample_size", "tier"]]
final_länsförsäkringar_df = final_länsförsäkringar_df.rename(
    columns={
        "client_imps": "Impressions", 
        "client_fixations": "Fixations",
        "client_inview": "Inviews",
        #"tobii_inview/impression_ratio": "tobii_Inview Ratio",
        #"tobii_fixation/impression_ratio": "Tobii Fixation Ratio",
        #"client_fixation/impression_ratio": "Fixation Ratio",
        "tobii_Fixation/Inview Ratio": "Fixation/Tobii Inview Ratio",
        "client_Fixation/Inview Ratio": "Fixation/Inview Ratio",
        "sample_size": "Sample Size"
    }
)

In [6]:
names = ["Jula", "Tele2", "Nordea", "Länsförsäkringar"]

with pd.ExcelWriter(f"../temp/1439_result_based_on_tier_with_weights_FINAL.xlsx") as writer:
    for i, df in enumerate([final_jula_df, final_tele2_df, final_nordea_df, final_länsförsäkringar_df]):
        sheet = names[i]
        df.to_excel(writer, sheet_name=sheet, index=False)

In [7]:
names = ["Jula", "Tele2", "Nordea", "Länsförsäkringar"]
final_jula_df = final_jula_df[["hostname", "Impressions", "Inviews", "client_Inview Ratio", "Fixation/Inview Ratio", "Fixation/Tobii Inview Ratio", "Sample Size"]]
final_jula_df = final_jula_df.rename(columns={"client_Inview Ratio": "Inview Ratio"})
final_jula_df = final_jula_df.sort_values(by=["hostname"])

final_tele2_df = final_tele2_df[["hostname", "Impressions", "Inviews", "client_Inview Ratio", "Fixation/Inview Ratio", "Fixation/Tobii Inview Ratio", "Sample Size"]]
final_tele2_df = final_tele2_df.rename(columns={"client_Inview Ratio": "Inview Ratio"})
final_tele2_df = final_tele2_df.sort_values(by=["hostname"])

final_nordea_df = final_nordea_df[["hostname", "Impressions", "Inviews", "client_Inview Ratio", "Fixation/Inview Ratio", "Fixation/Tobii Inview Ratio", "Sample Size"]]
final_nordea_df = final_nordea_df.rename(columns={"client_Inview Ratio": "Inview Ratio"})
final_nordea_df = final_nordea_df.sort_values(by=["hostname"])

final_länsförsäkringar_df = final_länsförsäkringar_df[["hostname", "Impressions", "Inviews", "client_Inview Ratio", "Fixation/Inview Ratio", "Fixation/Tobii Inview Ratio", "Sample Size"]]
final_länsförsäkringar_df = final_länsförsäkringar_df.rename(columns={"client_Inview Ratio": "Inview Ratio"})
final_länsförsäkringar_df = final_länsförsäkringar_df.sort_values(by=["hostname"])

with pd.ExcelWriter(f"../temp/schibsted_candidates/schibsted_results.xlsx") as writer:
    for i, df in enumerate([final_jula_df, final_tele2_df, final_nordea_df, final_länsförsäkringar_df]):
        sheet = names[i]
        df.to_excel(writer, sheet_name=sheet, index=False)

In [8]:
comments = ["Tier 1", "Tier 2", "Tier 3"]
names = ["Jula", "Tele2", "Nordea", "Länsförsäkringar"]

with pd.ExcelWriter(f"../temp/1439_result_FINAL.xlsx") as writer:
    for i, x in enumerate([jula_data, tele2_data, nordea_data, länsförsäkringar_data]):
        sheet = names[i]
        rows = 0
        spaces = 0
        for n, df in enumerate(x):
            pd.Series(comments[n]).to_excel(writer, sheet_name=sheet, index=False, header=False, startrow=rows + spaces)
            df.to_excel(writer, sheet, index=False, startrow=1 + rows + spaces)
            rows += len(df) + 2
            spaces += 2